### Запуск библиотек

In [1]:
!pip install pandas==1.3.5
!pip install pandas nltk
!pip install pymorphy3
!pip install nlpaug

In [2]:
#Для работы на TPU
#!pip install --upgrade tensorflow

In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pymorphy3
import random
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac
import hashlib
import nlpaug.augmenter.word as naw

# Загрузка необходимых ресурсов для токенизации на русском языке
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

### Создание словаря услуг

In [5]:
spr = pd.read_excel('datasets/spravochnik.xlsx')
spr

,service_id,ServiceCode,ServiceName
0,0,F01.01.01.1.001,"Прием (осмотр, консультация) врача-акушера-гин..."
1,1,F05.01.01.1.001,"Прием (осмотр, консультация) врача-гастроэнтер..."
2,2,F14.01.01.1.002,"Прием (осмотр, консультация) врача-кардиолога ..."
3,3,F31.05.01.1.001,Взятие крови из периферической вены
4,4,F32.01.01.1.001,"Прием (осмотр, консультация) врача-оторинолари..."
...,...,...,...
70,70,L18.41.02.0.001,Антистрептолизин-О (АСЛ-О)
71,71,L18.41.02.0.002,Ревматоидный фактор
72,72,L18.41.02.0.003,"С-реактивный белок (СРБ, CRP)"
73,73,L18.46.01.0.088,"Коронавирус COVID-19( SARS CoV-2), определение..."


In [6]:
#Данный код был необходим, чтобы добавить нулевую услугу,
#заменить им значение Nan
no_service = pd.DataFrame(columns=spr.columns)
no_service.at[1, 'service_id'] = 75
no_service.at[1, 'ServiceCode'] = 'None'
no_service.at[1, 'ServiceName'] = 'Нет услуг'

In [7]:
spr_new = spr.append(no_service, ignore_index=True)

spr_new.head()

<ipython-input-7-4d172c2895be>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  spr_new = spr.append(no_service, ignore_index=True)


,service_id,ServiceCode,ServiceName
0,0,F01.01.01.1.001,"Прием (осмотр, консультация) врача-акушера-гин..."
1,1,F05.01.01.1.001,"Прием (осмотр, консультация) врача-гастроэнтер..."
2,2,F14.01.01.1.002,"Прием (осмотр, консультация) врача-кардиолога ..."
3,3,F31.05.01.1.001,Взятие крови из периферической вены
4,4,F32.01.01.1.001,"Прием (осмотр, консультация) врача-оторинолари..."


In [8]:
spr_new.set_index('service_id', inplace=True)

# Преобразуем датафрейм в словарь
dictionary = spr_new.to_dict('index')


In [9]:
dictionary

{0: {'ServiceCode': 'F01.01.01.1.001',
  'ServiceName': 'Прием (осмотр, консультация) врача-акушера-гинеколога первичный'},
 1: {'ServiceCode': 'F05.01.01.1.001',
  'ServiceName': 'Прием (осмотр, консультация) врача-гастроэнтеролога первичный'},
 2: {'ServiceCode': 'F14.01.01.1.002',
  'ServiceName': 'Прием (осмотр, консультация) врача-кардиолога повторный'},
 3: {'ServiceCode': 'F31.05.01.1.001',
  'ServiceName': 'Взятие крови из периферической вены'},
 4: {'ServiceCode': 'F32.01.01.1.001',
  'ServiceName': 'Прием (осмотр, консультация) врача-оториноларинголога первичный'},
 5: {'ServiceCode': 'F48.01.00.1.001',
  'ServiceName': 'Прием (осмотр, консультация) врача-стоматолога первичный'},
 6: {'ServiceCode': 'F48.08.01.1.003',
  'ServiceName': 'Инфильтрационная анестезия (в стоматологии)'},
 7: {'ServiceCode': 'F48.56.01.1.006',
  'ServiceName': 'Снятие пигментированного налета (система Air-Flow, 1 зуб)'},
 8: {'ServiceCode': 'F48.56.01.1.012',
  'ServiceName': 'Гигиена полости рта и 

In [10]:
dictionary.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75])

### Создаем обработанный train

In [11]:
train_org = pd.read_excel('datasets/train.xlsx')
train_org.head()

,letter_id,guarantee_letter_text,service_ids_list,class
0,30624374,Ингосстрах \n \nF70.04.02.0.001 Магнитно-резон...,"[32, 18]",1
1,30637539,Согаз \n \nОрганизовать приемы врача (ревматол...,NaN,0
2,30546372,Согаз \n \nОрганизовать приемы врача терапе...,NaN,0
3,30718879,СОГАЗ \n \nОрганизовать приемы врача офталь...,NaN,0
4,30597751,"АО ""АльфаСтрахование"" г.Москва благодарит з...","[34, 39]",1


In [12]:
train_copy = train_org.copy()

In [13]:
# Замена NaN на 75 в столбце service_ids_list
train_copy['service_ids_list'] = train_copy['service_ids_list'].fillna('[75]')
train_copy.head()

,letter_id,guarantee_letter_text,service_ids_list,class
0,30624374,Ингосстрах \n \nF70.04.02.0.001 Магнитно-резон...,"[32, 18]",1
1,30637539,Согаз \n \nОрганизовать приемы врача (ревматол...,[75],0
2,30546372,Согаз \n \nОрганизовать приемы врача терапе...,[75],0
3,30718879,СОГАЗ \n \nОрганизовать приемы врача офталь...,[75],0
4,30597751,"АО ""АльфаСтрахование"" г.Москва благодарит з...","[34, 39]",1


In [14]:
#Датасет вручную нами переработанный, где в 1 строчке - 1 услуга
uslugi_orig = pd.read_excel('datasets/uslugi.xlsx')
uslugi_copy = uslugi_orig.copy()


In [15]:
uslugi_orig['service_ids_list'] = uslugi_orig['service_ids_list'].str.replace(r"[^\d\.]", "", regex=True)

In [16]:
# Определение функции для объединения строк
def merge_rows(df):
    #Объединяем письма - для расширения датасета
    dataframe_2 = pd.DataFrame(columns=df.columns)
    for i in range(len(df) - 1):
        # Выбор строк
        row_1 = df.iloc[i]
        row_2 = df.iloc[i + 1]

        # Создание нового letter_id (простой способ)
        new_letter_id = np.random.randint(10000000, 99999999)
        while new_letter_id in df['letter_id']:
            new_letter_id = new_letter_id

        # Объединение текстов писем
        combined_text = row_1['guarantee_letter_text'] + "." + row_2['guarantee_letter_text']

        # Объединение списков индентификаторов услуг
        #df['service_ids_list'] = df['service_ids_list'].astype
        combined_service_list = list(set(row_1['service_ids_list'] + row_2['service_ids_list']))
                # Определение класса
        combined_class = 1 if combined_service_list else 0

        dataframe_2 = dataframe_2.append({
            'letter_id': new_letter_id,
            'guarantee_letter_text': combined_text,
            'service_ids_list': combined_service_list,
            'class': combined_class
        }, ignore_index=True)

    return dataframe_2

    return new_df

In [17]:
df_new = merge_rows(uslugi_orig)

<ipython-input-16-732bcf773e2e>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe_2 = dataframe_2.append({
<ipython-input-16-732bcf773e2e>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe_2 = dataframe_2.append({
<ipython-input-16-732bcf773e2e>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe_2 = dataframe_2.append({
<ipython-input-16-732bcf773e2e>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe_2 = dataframe_2.append({
<ipython-input-16-732bcf773e2e>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataf

In [18]:
df_mixed = pd.concat([uslugi_orig, df_new], ignore_index = True)
df_mixed

,letter_id,guarantee_letter_text,service_ids_list,class
0,30906460,"Страховая Компания ПАО ""Группа Ренессанс Ст...",0,1
1,30906460,"Страховая Компания ПАО ""Группа Ренессанс Ст...",1,1
2,30858383,"АО ""СОГАЗ"" \n %СОГЛ% \nПрименение фторсодержащ...",10,1
3,30907300,"САО ""РЕСО-Гарантия"" гарантирует оплату пере...",11,1
4,30777882,"Страховая Компания ПАО ""Группа Ренессанс Ст...",12,1
...,...,...,...,...
160,80519903,"САО ""РЕСО-Гарантия"" гарантирует оплату пере...","[7, 2, 1]",1
161,31534072,"САО ""РЕСО-Гарантия"" гарантирует оплату пере...","[3, 7, 2]",1
162,12873511,Ингосстрах \n \nL18.46.01.0.088 Коронавирус CO...,"[3, 7, 4]",1
163,96885333,АО «СОГАЗ» \n \nПредоперационная подготовка: Г...,"[7, 4, 8]",1


In [19]:
# Убираем знаки - пробелы
def replace_probel(df):
  df = df.replace('\n',' ')
  return df

In [20]:
#Соединяем 2 датасета
train_concate = pd.concat([train_copy, df_mixed], ignore_index=True)
train_concate['guarantee_letter_text'] = train_concate['guarantee_letter_text'].apply(replace_probel)
train_concate

,letter_id,guarantee_letter_text,service_ids_list,class
0,30624374,Ингосстрах F70.04.02.0.001 Магнитно-резонан...,"[32, 18]",1
1,30637539,Согаз Организовать приемы врача (ревматолог...,[75],0
2,30546372,Согаз Организовать приемы врача терапевт...,[75],0
3,30718879,СОГАЗ Организовать приемы врача офтальмо...,[75],0
4,30597751,"АО ""АльфаСтрахование"" г.Москва благодарит з...","[34, 39]",1
...,...,...,...,...
322,80519903,"САО ""РЕСО-Гарантия"" гарантирует оплату пере...","[7, 2, 1]",1
323,31534072,"САО ""РЕСО-Гарантия"" гарантирует оплату пере...","[3, 7, 2]",1
324,12873511,Ингосстрах L18.46.01.0.088 Коронавирус COVI...,"[3, 7, 4]",1
325,96885333,АО «СОГАЗ» Предоперационная подготовка: Гос...,"[7, 4, 8]",1


### Вытаскиваем ServiceCode

In [21]:
# Смотрим SetciveCode и вытаскиваем его из основного письма, т.к. при аугментации он нам помешает
ServiceCode = []
for i in dictionary:
    if dictionary[i]['ServiceCode'] not in ServiceCode:
        ServiceCode.append(dictionary[i]['ServiceCode'])
ServiceCode

['F01.01.01.1.001',
 'F05.01.01.1.001',
 'F14.01.01.1.002',
 'F31.05.01.1.001',
 'F32.01.01.1.001',
 'F48.01.00.1.001',
 'F48.08.01.1.003',
 'F48.56.01.1.006',
 'F48.56.01.1.012',
 'F48.56.02.1.003',
 'F48.56.03.1.013',
 'F51.01.01.1.001',
 'F51.01.01.1.002',
 'F55.27.01.1.002',
 'F55.27.02.1.009',
 'F55.27.07.1.007',
 'F55.27.08.1.003',
 'F55.27.08.1.005',
 'F55.27.08.1.023',
 'F56.01.01.1.002',
 'F57.05.01.1.054',
 'F59.09.01.1.001',
 'F59.09.01.1.013',
 'F59.09.01.1.015',
 'F59.09.02.1.001',
 'F64.05.01.0.003',
 'F64.05.01.0.006',
 'F64.05.02.0.005',
 'F69.18.02.0.002',
 'F69.18.02.0.005',
 'F69.20.02.0.004',
 'F69.20.03.0.002',
 'F70.04.02.0.001',
 'F70.04.02.0.007',
 'F70.04.04.0.006',
 'F70.04.05.0.006',
 'F70.04.09.0.001',
 'F70.04.09.0.002',
 'F70.04.09.0.003',
 'F70.04.09.0.004',
 'F70.04.09.0.006',
 'F70.04.09.0.007',
 'F70.04.09.0.008',
 'F73.05.01.1.006',
 'L18.30.00.0.001',
 'L18.30.00.0.005',
 'L18.30.00.0.007',
 'L18.31.01.0.001',
 'L18.31.06.0.001',
 'L18.31.06.0.004',


In [22]:
def extract_service_code(stroca:str):
    code_list = []
    for code in ServiceCode:
        if stroca.find(code) != -1:
            code_list.append(code)
    #code_str = (', '.join(str(el) for el in code_list))
    return code_list #code_str

In [23]:
def remove_service_code(stroca:str):
    for code in ServiceCode:
        if stroca.find(code) != -1:
            stroca = stroca.replace(code, '')
    return stroca

In [24]:
train_concate['ServiceCode'] = train_concate['guarantee_letter_text'].apply(extract_service_code)
train_concate['guarantee_letter_text'] = train_concate['guarantee_letter_text'].apply(remove_service_code)

In [25]:
train_aug = train_concate.copy()
train_aug.head()

,letter_id,guarantee_letter_text,service_ids_list,class,ServiceCode
0,30624374,Ингосстрах Магнитно-резонансная томография...,"[32, 18]",1,"[F55.27.08.1.023, F70.04.02.0.001]"
1,30637539,Согаз Организовать приемы врача (ревматолог...,[75],0,[]
2,30546372,Согаз Организовать приемы врача терапевт...,[75],0,[]
3,30718879,СОГАЗ Организовать приемы врача офтальмо...,[75],0,[]
4,30597751,"АО ""АльфаСтрахование"" г.Москва благодарит з...","[34, 39]",1,"[F70.04.04.0.006, F70.04.09.0.004]"


### Аугментация для бинарной модели (код письма)

In [26]:
import nlpaug.augmenter.word.context_word_embs as nawcwe
import nlpaug.augmenter.word.word_embs as nawwe
import nlpaug.augmenter.word.spelling as naws

In [27]:
# AUGMENT_TEXT
# Функция для аугментации текста - SynonymAug
def Synonym_Aug(text, aug_p=0.3):
    # SyninymAug - здесь можно взять словарь для перефразирований из ppdb или др.
    augmenter = naw.SynonymAug(
        aug_p=aug_p)#, aug_src='ppdb', model_path='/content/drive/MyDrive/HaK_MEDCI/dict/ppdb-1.0-m-phrasal.gz'


    # Аугментация текста
    augmented_text = augmenter.augment(text, n=1)

    return augmented_text

# Количество аугментированных текстов, например, в 2/5/10/100 раз
augmentation_factor = 2

# Создание нового датасета с аугментированными данными
augmented_data = []
for _, row in train_aug.iterrows():
    for _ in range(augmentation_factor):
        augmented_text = Synonym_Aug(row['guarantee_letter_text'])
        augmented_data.append(
            {'letter_id': len(augmented_data),
             'guarantee_letter_text': ' '.join(augmented_text),
             'service_ids_list':row['service_ids_list'],
             'ServiceCode':row['ServiceCode'],
             'class': row['class']})

train_Synonym = pd.DataFrame(augmented_data)
train_Synonym.head()

,letter_id,guarantee_letter_text,service_ids_list,ServiceCode,class
0,0,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
1,1,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
2,2,"Согаз Организовать приемы врача (ревматолог ),...",[75],[],0
3,3,"Согаз Организовать приемы врача (ревматолог ),...",[75],[],0
4,4,"Согаз Организовать приемы врача терапевта, нео...",[75],[],0


In [28]:
train_Synonym.shape

(654, 5)

In [29]:
# AUGMENT_TEXT
# Функция для аугментации текста - RandomWord
def RandomWord_Aug(text, aug_p=0.3):

    augmenter = naw.RandomWordAug(aug_p=aug_p, action = 'substitute')

    # Аугментация текста
    augmented_text = augmenter.augment(text, n=1)

    return augmented_text

# Количество аугментированных текстов, например, в 2/5/10/100 раз
augmentation_factor = 5

# Создание нового датасета с аугментированными данными
augmented_data = []
for _, row in train_Synonym.iterrows():
    for _ in range(augmentation_factor):
        augmented_text = RandomWord_Aug(row['guarantee_letter_text'])
        augmented_data.append(
            {'letter_id': len(augmented_data),
             'guarantee_letter_text': ' '.join(augmented_text),
             'service_ids_list':row['service_ids_list'],
             'ServiceCode':row['ServiceCode'],
             'class': row['class']})

train_Synonym_RandomWord = pd.DataFrame(augmented_data)
train_Synonym_RandomWord.head()

,letter_id,guarantee_letter_text,service_ids_list,ServiceCode,class
0,0,Ингосстрах Магнитно - _ томография головного м...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
1,1,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
2,2,Ингосстрах _ - _ томография головного мозга 1 ...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
3,3,_ Магнитно - резонансная _ головного мозга 1 _...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
4,4,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1


In [30]:
train_Synonym_RandomWord.shape

(3270, 5)

In [31]:
# AUGMENT_TEXT
# Функция для аугментации текста - Spelling
def Spelling_Aug(text, aug_p=0.3):

    augmenter = naw.SpellingAug(aug_p=aug_p)

    # Аугментация текста
    augmented_text = augmenter.augment(text, n=1)

    return augmented_text

# Количество аугментированных текстов, например, в 2/5/10/100 раз
augmentation_factor = 3

# Создание нового датасета с аугментированными данными
augmented_data = []
for _, row in train_Synonym_RandomWord.iterrows():
    for _ in range(augmentation_factor):
        augmented_text = Spelling_Aug(row['guarantee_letter_text'])
        augmented_data.append(
            {'letter_id': len(augmented_data),
             'guarantee_letter_text': ' '.join(augmented_text),
             'service_ids_list':row['service_ids_list'],
             'ServiceCode':row['ServiceCode'],
             'class': row['class']})

train_Synonym_RandomWord_Spelling = pd.DataFrame(augmented_data)
train_Synonym_RandomWord_Spelling.head()

,letter_id,guarantee_letter_text,service_ids_list,ServiceCode,class
0,0,Ингосстрах Магнитно - _ томография головного м...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
1,1,Ингосстрах Магнитно - _ томография головного м...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
2,2,Ингосстрах Магнитно - _ томография головного м...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
3,3,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
4,4,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1


In [32]:
train_Synonym_RandomWord_Spelling.shape

(9810, 5)

In [34]:
train_Synonym_RandomWord_Spelling['guarantee_letter_text'][0]

'Ингосстрах Магнитно - _ томография головного мозга 10 Дуплексное сканирование экстракраниальных отделов брахиоцефальных артерий 10 на Белорусской Оплата будет осуществляться в соответствии с условиями Договора и Программой, являющейся приложением к Договору. Оплата медицинских _, оказанных по данному гарантийному письму, может _ оспорена по результатам экспертизы _ и / или медицинской документации. Диагноз: _ _, синдром когнитивных _, декомпенсация, уточнить генез. Исключить гемодинамически значимые _, кинкинг, койлинг БЦА. Исключить пароксизмальную _ коры головного мозга. Исключить интракраниальное объемное образование. (G93. _ )'

In [35]:
train_Synonym_RandomWord_Spelling['guarantee_letter_text'][4]

'Ингосстрах Магнитно - резонансная томография головного мозга 10 Дуплексное сканирование _ _ брахиоцефальных артерий 13 на Белорусской Оплата будет осуществляться в соответствии с условиями _ и Программой, являющейся приложением к Договору. _ медицинских услуг, оказанных по данному гарантийному _, может быть оспорена по результатам экспертизы счета и / _ медицинской документации. Диагноз: Вестибулоатактический синдром, _ когнитивных нарушений, декомпенсация, уточнить генез. Исключить гемодинамически значимые стенозы, кинкинг, койлинг БЦА. _ пароксизмальную _ коры головного мозга. _ интракраниальное объемное образование. (G93. 4th )'

In [36]:
# Добавляем все датасеты в один
aug_df = pd.concat(
    [train_aug, train_Synonym,
     train_Synonym_RandomWord,
     train_Synonym_RandomWord_Spelling], ignore_index=True)
aug_df

,letter_id,guarantee_letter_text,service_ids_list,class,ServiceCode
0,30624374,Ингосстрах Магнитно-резонансная томография...,"[32, 18]",1,"[F55.27.08.1.023, F70.04.02.0.001]"
1,30637539,Согаз Организовать приемы врача (ревматолог...,[75],0,[]
2,30546372,Согаз Организовать приемы врача терапевт...,[75],0,[]
3,30718879,СОГАЗ Организовать приемы врача офтальмо...,[75],0,[]
4,30597751,"АО ""АльфаСтрахование"" г.Москва благодарит з...","[34, 39]",1,"[F70.04.04.0.006, F70.04.09.0.004]"
...,...,...,...,...,...
14056,9805,"Страховая Компания ПАО "" Группа Ренессанс Стра...","[8, 9]",1,[]
14057,9806,"Страховая Компания ПАО "" Группа Ренессанс Стра...","[8, 9]",1,[]
14058,9807,"Страховая Компания _ "" Группа Ренессанс Страхо...","[8, 9]",1,[]
14059,9808,"Страховая Компания _ "" Группа Ренессанс Страхо...","[8, 9]",1,[]


In [37]:
# Перемешиваем предложения.
import nlpaug.augmenter.sentence as nas
# AUGMENT_TEXT
# Функция для аугментации текста - RandomSentAug
def RandomSentAug(text, aug_p=0.3):
    # RandomSentAug
    augmenter = aug = nas.RandomSentAug()

    # Аугментация текста
    augmented_text = augmenter.augment(text, n=1)

    return augmented_text

# Количество аугментированных текстов, например, в 2/5/10/100 раз
augmentation_factor = 3

# Создание нового датасета с аугментированными данными
augmented_data = []
for _, row in aug_df.iterrows():
    for _ in range(augmentation_factor):
        augmented_text = RandomSentAug(row['guarantee_letter_text'])
        augmented_data.append(
            {'letter_id': len(augmented_data),
             'guarantee_letter_text': ' '.join(augmented_text),
             'service_ids_list':row['service_ids_list'],
             'ServiceCode':row['ServiceCode'],
             'class': row['class']})

aug_df_reverseSet = pd.DataFrame(augmented_data)
aug_df_reverseSet.head()

,letter_id,guarantee_letter_text,service_ids_list,ServiceCode,class
0,0,Ингосстрах Магнитно-резонансная томография...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
1,1,"Оплата медицинских услуг, оказанных по данному...","[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
2,2,"Оплата медицинских услуг, оказанных по данному...","[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
3,3,"Услуги, оказанные без медицинских показаний, ...",[75],[],0
4,4,"Услуги, оказанные без медицинских показаний, ...",[75],[],0


In [38]:
aug_df_reverseSet

,letter_id,guarantee_letter_text,service_ids_list,ServiceCode,class
0,0,Ингосстрах Магнитно-резонансная томография...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
1,1,"Оплата медицинских услуг, оказанных по данному...","[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
2,2,"Оплата медицинских услуг, оказанных по данному...","[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
3,3,"Услуги, оказанные без медицинских показаний, ...",[75],[],0
4,4,"Услуги, оказанные без медицинских показаний, ...",[75],[],0
...,...,...,...,...,...
42178,42178,"Страховая Компания _ "" Группа Ренессанс Страхо...","[8, 9]",[],1
42179,42179,"07. Страховая Компания _ "" Группа Ренессанс Ст...","[8, 9]",[],1
42180,42180,"2024г. Страховая Компания _ "" Группа Ренессанс...","[8, 9]",[],1
42181,42181,"Страховая Компания _ "" Группа Ренессанс Страхо...","[8, 9]",[],1


In [39]:
#приводить слово к нормальной форме
#ставить слово в нужную форму. Например, ставить слово во множественное число, менять падеж слова и т.д.
#возвращать грамматическую информацию о слове (число, род, падеж, часть речи и т.д.)

import pymorphy3

#MORHANALYZER
# Инициализация PyMorphy3
morph = pymorphy3.MorphAnalyzer()

# Функция для расширения текста новыми словами на русском языке
def expand_text_with_words(text, num_words_to_add=10):
    # Токенизация текста на слова
    tokens = word_tokenize(text, language='russian')

    # Исключение стоп-слов из русского корпуса
    russian_stopwords = set(stopwords.words('russian'))
    tokens_filtered = [word for word in tokens if word not in russian_stopwords]

    # Лемматизация слов с использованием PyMorphy3
    lemmas = [morph.parse(word)[0].normal_form for word in tokens_filtered]

    # Выбор уникальных слов, не входящих в исходный текст
    new_words = random.sample(lemmas, min(num_words_to_add, len(lemmas)))

    # Добавление новых слов к тексту
    expanded_text = ' '.join(tokens + new_words)

    return expanded_text

In [40]:
aug_df_reverseSet['guarantee_letter_text'] = aug_df_reverseSet['guarantee_letter_text'].apply(expand_text_with_words)
aug_df_reverseSet

,letter_id,guarantee_letter_text,service_ids_list,ServiceCode,class
0,0,Ингосстрах Магнитно-резонансная томография гол...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
1,1,"Оплата медицинских услуг , оказанных по данном...","[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
2,2,"Оплата медицинских услуг , оказанных по данном...","[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
3,3,"Услуги , оказанные без медицинских показаний ,...",[75],[],0
4,4,"Услуги , оказанные без медицинских показаний ,...",[75],[],0
...,...,...,...,...,...
42178,42178,Страховая Компания _ `` Группа Ренессанс Страх...,"[8, 9]",[],1
42179,42179,07 . Страховая Компания _ `` Группа Ренессанс ...,"[8, 9]",[],1
42180,42180,2024г . Страховая Компания _ `` Группа Ренесса...,"[8, 9]",[],1
42181,42181,Страховая Компания _ `` Группа Ренессанс Страх...,"[8, 9]",[],1


In [41]:
# Сохранение нового датасета
#aug_df_reverseSet.to_csv('datasets/final_aug_df.csv', index=False)

In [42]:
#final_aug_df = pd.read_csv('datasets/final_aug_df.csv')
#final_aug_df

,letter_id,guarantee_letter_text,service_ids_list,ServiceCode,class
0,0,Ингосстрах Магнитно-резонансная томография гол...,"[32, 18]","['F55.27.08.1.023', 'F70.04.02.0.001']",1
1,1,"Оплата медицинских услуг , оказанных по данном...","[32, 18]","['F55.27.08.1.023', 'F70.04.02.0.001']",1
2,2,"Оплата медицинских услуг , оказанных по данном...","[32, 18]","['F55.27.08.1.023', 'F70.04.02.0.001']",1
3,3,"Услуги , оказанные без медицинских показаний ,...",[75],[],0
4,4,"Услуги , оказанные без медицинских показаний ,...",[75],[],0
...,...,...,...,...,...
42178,42178,Страховая Компания _ `` Группа Ренессанс Страх...,"['8', '9']",[],1
42179,42179,07 . Страховая Компания _ `` Группа Ренессанс ...,"['8', '9']",[],1
42180,42180,2024г . Страховая Компания _ `` Группа Ренесса...,"['8', '9']",[],1
42181,42181,Страховая Компания _ `` Группа Ренессанс Страх...,"['8', '9']",[],1


### Аугментация для мультиклассовой модели (по кодам услуг)

In [43]:
spr_class_1 = pd.read_excel('datasets/spravochnik.xlsx')
spr_class_1.set_index('service_id', inplace=True)

# Преобразуем датафрейм в словарь
dictionary_class_1 = spr_class_1.to_dict('index')

Сделаем словарь стоп слов для ru и en словарей

In [44]:
stop_list_ru = []

for i in range(len(spr_class_1['ServiceName'])-1):
  stop_list_ru.append(spr_class_1['ServiceName'][i].split())

stop_list_words_ru = []

for i in range(len(stop_list_ru)-1):
  stop_list_words_ru += stop_list_ru[i]

stop_list_words_ru

['Прием',
 '(осмотр,',
 'консультация)',
 'врача-акушера-гинеколога',
 'первичный',
 'Прием',
 '(осмотр,',
 'консультация)',
 'врача-гастроэнтеролога',
 'первичный',
 'Прием',
 '(осмотр,',
 'консультация)',
 'врача-кардиолога',
 'повторный',
 'Взятие',
 'крови',
 'из',
 'периферической',
 'вены',
 'Прием',
 '(осмотр,',
 'консультация)',
 'врача-оториноларинголога',
 'первичный',
 'Прием',
 '(осмотр,',
 'консультация)',
 'врача-стоматолога',
 'первичный',
 'Инфильтрационная',
 'анестезия',
 '(в',
 'стоматологии)',
 'Снятие',
 'пигментированного',
 'налета',
 '(система',
 'Air-Flow,',
 '1',
 'зуб)',
 'Гигиена',
 'полости',
 'рта',
 'и',
 'зубов',
 '(обработка',
 'антисептиком)',
 'Ультразвуковое',
 'удаление',
 'наддесневых',
 'и',
 'поддесневых',
 'зубных',
 'отложений',
 'в',
 'области',
 'зуба',
 'Применение',
 'фторсодержащих',
 'препаратов',
 '(1',
 'зуб)',
 'Прием',
 '(осмотр,',
 'консультация)',
 'врача-терапевта',
 'первичный',
 'Прием',
 '(осмотр,',
 'консультация)',
 'врача-тер

In [45]:
import re
import string
string.punctuation

def remove_punctuation(text):
    return ''.join([ch for ch in text if ch not in
    string.punctuation]) # метод для удаления пунктуации

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])
    # метод для удаления чисел

def remove_space(text):
    return re.sub(r'\s+','  ', text, flags=re.I)
    return re.sub(r'\s+',' ', text, flags=re.I)
    # метод для удаления пробелов

stop_list_ru = []
stop_list_words_ru = list(set(stop_list_words_ru))
for i in stop_list_words_ru:
    if i is not None or i=='  ':
        i = remove_punctuation(i)
        i = remove_numbers(i)
        i = remove_space(i)
        stop_list_ru.append(i)

In [46]:
stop_list_ru

['яиц',
 'врачакардиолога',
 'Магнитнорезонансная',
 'из',
 'при',
 'давления',
 'без',
 'а',
 'Рентгенография',
 'исследование',
 'Аспартатаминотрансферазы',
 '  ти',
 'ПТ',
 'кислота',
 'СРБ',
 'Глюкоза',
 'Гаммаглютамилтрансфераза',
 'мл',
 'анализ',
 'Эхокардиография',
 '  ',
 'методом',
 'крови',
 'Триглицериды',
 'анестезия',
 'Индекс',
 'артериального',
 'иглотерапия',
 'АСТ',
 'антисептиком',
 'врачауролога',
 'Белковые',
 'ЛПНП',
 'ЩФ',
 'брюшной',
 'скорости',
 'консультация',
 'пузырь',
 'проекции',
 'суставов',
 '  ',
 'вены',
 'гельминтов',
 'полости',
 'фосфатаза',
 'лейкоцитарной',
 'Мочевая',
 'часов',
 'легких',
 'Холтеровское',
 'Эндоскопический',
 'мониторирование',
 'поля',
 'магнитнорезонансной',
 'врачагастроэнтеролога',
 'АЛТ',
 'Щелочная',
 'эритроцитов',
 'Среактивный',
 'зуб',
 'точки',
 'система',
 'Гигиена',
 'области',
 'Альфаамилаза',
 'возможностью',
 'Холестерин',
 'классическая',
 'врачатерапевта',
 '  ',
 'контрастного',
 'AirFlow',
 'pylori',
 'Ректос

In [47]:
# Убираем письма без услуг
train_class1 = train_concate[train_concate['class']==1]
train_class1 = train_class1.reset_index()
train_class1 = train_class1.drop(columns=['ServiceCode','index'])
print(train_class1.shape)
train_class1.head()

(283, 4)


,letter_id,guarantee_letter_text,service_ids_list,class
0,30624374,Ингосстрах Магнитно-резонансная томография...,"[32, 18]",1
1,30597751,"АО ""АльфаСтрахование"" г.Москва благодарит з...","[34, 39]",1
2,30861093,"АО ""СОГАЗ"" УЗИ молочных желез Гарантируем оп...",[15],1
3,30858383,"АО ""СОГАЗ"" %СОГЛ% Ультразвуковое удаление н...","[9, 41, 6, 10]",1
4,30644434,АО «СОГАЗ» %СОГЛ% Кальпротектин (в кале). ...,"[41, 49]",1


In [48]:
train_aug_or = train_class1.copy()
train_aug_or.head()

,letter_id,guarantee_letter_text,service_ids_list,class
0,30624374,Ингосстрах Магнитно-резонансная томография...,"[32, 18]",1
1,30597751,"АО ""АльфаСтрахование"" г.Москва благодарит з...","[34, 39]",1
2,30861093,"АО ""СОГАЗ"" УЗИ молочных желез Гарантируем оп...",[15],1
3,30858383,"АО ""СОГАЗ"" %СОГЛ% Ультразвуковое удаление н...","[9, 41, 6, 10]",1
4,30644434,АО «СОГАЗ» %СОГЛ% Кальпротектин (в кале). ...,"[41, 49]",1


Аугментация на основе машинного перевода методом Back Translation дает довольно плохие результаты - искаженный перевод медицинских терминов и компаний(хотя компании можно убрать). В дальнейшей английский текст хорошо бы работал с такими методами как: WordEmbsAug, ContextualWordEmbsAug, AntonymAug, SynonymAug (которые используют англоязычные словари и модели), - однако в виде плохих аугментированных текстов решено не брать данные методы для аугментации. В них же использовались исключение stopword (которые выше были сделаны для русского и английского языка)

In [49]:
# AUGMENT_TEXT
# Функция для аугментации текста - RandomWord_delete
def RandomWord_Aug_del(text, aug_p=0.3):
    """Функция для удаления случайных слов"""

    augmenter = naw.RandomWordAug(aug_p=aug_p, action = 'delete')
    # Аугментация текста
    augmented_text = augmenter.augment(text, n=1)

    return augmented_text

def RandomWord_Aug_swap(text, aug_p=0.3):
    """Функция для изменения расположения слов"""

    augmenter = naw.RandomWordAug(aug_p=aug_p, action = 'swap')
    augmented_text = augmenter.augment(text, n=1)

    return augmented_text

# Количество аугментированных текстов, например, в 2/5/10/100 раз
augmentation_factor = 5

# Создание нового датасета с аугментированными данными
augmented_data = []
for _, row in train_aug_or.iterrows():
    for _ in range(augmentation_factor):
        augmented_text = RandomWord_Aug_del(row['guarantee_letter_text'])
        augmented_data.append(
            {'letter_id': len(augmented_data),
             'guarantee_letter_text': ' '.join(augmented_text),
             'service_ids_list':row['service_ids_list'],
             'class': row['class']})

train_RandomWord_del = pd.DataFrame(augmented_data)
train_RandomWord_del.head()

,letter_id,guarantee_letter_text,service_ids_list,class
0,0,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]",1
1,1,Магнитно - резонансная томография головного мо...,"[32, 18]",1
2,2,Ингосстрах - резонансная томография головного ...,"[32, 18]",1
3,3,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]",1
4,4,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]",1


In [50]:
train_RandomWord_del.shape

(1415, 4)

In [51]:
#Используем RandomWord_Aug_swap
augmentation_factor = 3

augmented_data = []
for _, row in train_RandomWord_del.iterrows():
    for _ in range(augmentation_factor):
        augmented_text = RandomWord_Aug_swap(row['guarantee_letter_text'])
        augmented_data.append(
            {'letter_id': len(augmented_data),
             'guarantee_letter_text': ' '.join(augmented_text),
             'service_ids_list':row['service_ids_list'],
             'class': row['class']})

train_RandomWord_del_swap = pd.DataFrame(augmented_data)
print(train_RandomWord_del_swap.shape)
train_RandomWord_del_swap.head()

(4245, 4)


,letter_id,guarantee_letter_text,service_ids_list,class
0,0,Ингосстрах Магнитно - резонансная головного то...,"[32, 18]",1
1,1,Ингосстрах Магнитно - томография резонансная г...,"[32, 18]",1
2,2,Магнитно ингосстрах - резонансная томография г...,"[32, 18]",1
3,3,Магнитно - резонансная томография головного мо...,"[32, 18]",1
4,4,Магнитно - резонансная головного томография мо...,"[32, 18]",1


In [52]:
# Функция для аугментации текста - Spelling
def Spelling_Aug(text, aug_p=0.3):
    """Функция для внедрения в текст
      случайных орфографических ошибок"""

    augmenter = naw.SpellingAug(aug_p=aug_p)
    augmented_text = augmenter.augment(text, n=1)

    return augmented_text

augmentation_factor = 5

# Создание нового датасета с аугментированными данными на начальном датасете
augmented_data = []
for _, row in train_aug_or.iterrows():
    for _ in range(augmentation_factor):
        augmented_text = Spelling_Aug(row['guarantee_letter_text'])
        augmented_data.append(
            {'letter_id': len(augmented_data),
             'guarantee_letter_text': ' '.join(augmented_text),
             'service_ids_list':row['service_ids_list'],
             'class': row['class']})

train_Spelling = pd.DataFrame(augmented_data)
print(train_Spelling.shape)
train_Spelling.head()

(1415, 4)


,letter_id,guarantee_letter_text,service_ids_list,class
0,0,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]",1
1,1,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]",1
2,2,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]",1
3,3,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]",1
4,4,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]",1


Создание датасета

In [53]:
aug_df_class1 = pd.concat(
    [train_aug_or,
     train_Spelling,
     train_RandomWord_del_swap
     ], ignore_index=True)
aug_df_class1

,letter_id,guarantee_letter_text,service_ids_list,class
0,30624374,Ингосстрах Магнитно-резонансная томография...,"[32, 18]",1
1,30597751,"АО ""АльфаСтрахование"" г.Москва благодарит з...","[34, 39]",1
2,30861093,"АО ""СОГАЗ"" УЗИ молочных желез Гарантируем оп...",[15],1
3,30858383,"АО ""СОГАЗ"" %СОГЛ% Ультразвуковое удаление н...","[9, 41, 6, 10]",1
4,30644434,АО «СОГАЗ» %СОГЛ% Кальпротектин (в кале). ...,"[41, 49]",1
...,...,...,...,...
5938,4240,"Страховая Компания ПАО "" Группа Ренессанс Стра...","[8, 9]",1
5939,4241,"Страховая Компания ПАО "" Группа Ренессанс Стра...","[8, 9]",1
5940,4242,"Страховая Компания ПАО "" Группа Ренессанс Стра...","[8, 9]",1
5941,4243,"Страховая Компания "" ПАО Ренессанс Группа Стра...","[8, 9]",1


In [54]:
# Перемешиваем предложения.
import nlpaug.augmenter.sentence as nas

# Функция для аугментации текста - RandomSentAug
def RandomSentAug(text, aug_p=0.3):
    # Функция для перестановки предложений в тексте"""

    augmenter = nas.RandomSentAug()
    augmented_text = augmenter.augment(text, n=1)

    return augmented_text

augmentation_factor = 5

# Создание нового датасета с аугментированными данными
augmented_data = []
for _, row in aug_df_class1.iterrows():
    for _ in range(augmentation_factor):
        augmented_text = RandomSentAug(row['guarantee_letter_text'])
        augmented_data.append(
            {'letter_id': len(augmented_data),
             'guarantee_letter_text': ' '.join(augmented_text),
             'service_ids_list':row['service_ids_list'],
             'class': row['class']})

aug_df_multi_reverseSet = pd.DataFrame(augmented_data)
print(aug_df_reverseSet.shape)
aug_df_multi_reverseSet.head()

(42183, 5)


,letter_id,guarantee_letter_text,service_ids_list,class
0,0,"Оплата медицинских услуг, оказанных по данному...","[32, 18]",1
1,1,(G93.4) Диагноз: Вестибулоатактический синдром...,"[32, 18]",1
2,2,Ингосстрах Магнитно-резонансная томография...,"[32, 18]",1
3,3,"Оплата медицинских услуг, оказанных по данному...","[32, 18]",1
4,4,"Оплата медицинских услуг, оказанных по данному...","[32, 18]",1


In [55]:
#приводить слово к нормальной форме
#ставить слово в нужную форму. Например, ставить слово во множественное число, менять падеж слова и т.д.
#возвращать грамматическую информацию о слове (число, род, падеж, часть речи и т.д.)

import pymorphy3
morph = pymorphy3.MorphAnalyzer()

# Функция для расширения текста новыми словами на русском языке
def expand_text_with_words(text, num_words_to_add=10):
    # Токенизация текста на слова
    tokens = word_tokenize(text, language='russian')

    # Исключение стоп-слов из русского корпуса
    russian_stopwords = set(stopwords.words('russian'))
    tokens_filtered = [word for word in tokens if word not in russian_stopwords]

    # Лемматизация слов с использованием PyMorphy3
    lemmas = [morph.parse(word)[0].normal_form for word in tokens_filtered]

    # Выбор уникальных слов, не входящих в исходный текст
    new_words = random.sample(lemmas, min(num_words_to_add, len(lemmas)))

    # Добавление новых слов к тексту
    expanded_text = ' '.join(tokens + new_words)

    return expanded_text

In [56]:
aug_df_multi_reverseSet['guarantee_letter_text'] = aug_df_multi_reverseSet['guarantee_letter_text'].apply(expand_text_with_words)
aug_df_multi_reverseSet

,letter_id,guarantee_letter_text,service_ids_list,class
0,0,"Оплата медицинских услуг , оказанных по данном...","[32, 18]",1
1,1,( G93.4 ) Диагноз : Вестибулоатактический синд...,"[32, 18]",1
2,2,Ингосстрах Магнитно-резонансная томография гол...,"[32, 18]",1
3,3,"Оплата медицинских услуг , оказанных по данном...","[32, 18]",1
4,4,"Оплата медицинских услуг , оказанных по данном...","[32, 18]",1
...,...,...,...,...
29710,29710,07 Гарантируем . оплату медицинских услуг в со...,"[8, 9]",1
29711,29711,Страховая Компания ПАО `` Ренессанс Группа Стр...,"[8, 9]",1
29712,29712,07 Гарантируем . Страховая Компания ПАО `` Рен...,"[8, 9]",1
29713,29713,"Услуги , оказанные без медицинских показаний ,...","[8, 9]",1


In [57]:
# Сохранение нового датасета
#aug_df_multi_reverseSet.to_csv('datasets/multiclass_df.csv', index=False)

In [58]:
#multi_aug_df = pd.read_csv('datasets/multiclass_df.csv')
#multi_aug_df

,letter_id,guarantee_letter_text,service_ids_list,class
0,0,"Оплата медицинских услуг , оказанных по данном...","[32, 18]",1
1,1,( G93.4 ) Диагноз : Вестибулоатактический синд...,"[32, 18]",1
2,2,Ингосстрах Магнитно-резонансная томография гол...,"[32, 18]",1
3,3,"Оплата медицинских услуг , оказанных по данном...","[32, 18]",1
4,4,"Оплата медицинских услуг , оказанных по данном...","[32, 18]",1
...,...,...,...,...
29710,29710,07 Гарантируем . оплату медицинских услуг в со...,"['8', '9']",1
29711,29711,Страховая Компания ПАО `` Ренессанс Группа Стр...,"['8', '9']",1
29712,29712,07 Гарантируем . Страховая Компания ПАО `` Рен...,"['8', '9']",1
29713,29713,"Услуги , оказанные без медицинских показаний ,...","['8', '9']",1
